#  T-tests and P-values

In statistics, t-test is used to test if two data samples have a significant difference between their means. There are two types of t-test:

* **Student's t-test** (a.k.a. independent or uncorrelated t-test). This type of t-test is to compare the samples of **two independent populations** (e.g. test scores of students in two different classes). `scipy` provides the [`ttest_ind`](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.ttest_ind.html) method to conduct student's t-test.

* **Paired t-test** (a.k.a. dependent or correlated t-test). This type of t-test is to compare the samples of **the same population** (e.g. scores of different tests of students in the same class). `scipy` provides the [`ttest_rel`](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.ttest_rel.html) method to conduct paired t-test.

Both types of t-tests return a number which is called the **p-value**. If p-value is below 0.05, we can confidently declare the null-hypothesis is rejected and the difference is significant. If p-value is between 0.05 and 0.1, we may also declare the null-hypothesis is rejected but we are not highly confident. If p-value is above 0.1 we do not reject the null-hypothesis.

Read more about the t-test in [this article](http://b.link/test50) and [this Quora](http://b.link/unpaired97). Make sure you understand when to use which type of t-test. 

In [41]:
# Import libraries
import pandas as pd
from scipy.stats import ttest_ind
from io import StringIO
from scipy.stats import ttest_rel


### One tailed t-test 
In a packing plant, a machine packs cartons with jars. It is supposed that a new machine will pack faster on the average than the machine currently used. To test that hypothesis, the times it takes each machine to pack ten cartons are recorded. The results, in seconds, are shown in the tables in the file files_for_lab/ttest_machine.xlsx. Assume that there is sufficient evidence to conduct the t test, does the data provide sufficient evidence to show if one machine is better than the other?

#### Import dataset

In [15]:
data = pd.read_csv("ttest_machine.txt")
data.columns

Index(['New_machine Old_machine'], dtype='object')

In [16]:
data  # for some reason both names and data are on the same column

,New_machine Old_machine
0,42.1 42.7
1,41 43.6
2,41.3 43.8
3,41.8 43.3
4,42.4 42.5
5,42.8 43.5
6,43.2 43.1
7,42.3 41.7
8,41.8 44
9,42.7 44.1


In [17]:
data[['New_machine', 'Old_machine']] = data['New_machine Old_machine'].str.split(' ', expand=True)
data['New_machine'] = pd.to_numeric(data['New_machine'])
data['Old_machine'] = pd.to_numeric(data['Old_machine'])
data = data.drop('New_machine Old_machine', axis=1)

In [18]:
# Your code here:
current_machine_times = data["Old_machine"]
new_machine_times = data["New_machine"]

In [19]:
data

,New_machine,Old_machine
0,42.1,42.7
1,41.0,43.6
2,41.3,43.8
3,41.8,43.3
4,42.4,42.5
5,42.8,43.5
6,43.2,43.1
7,42.3,41.7
8,41.8,44.0
9,42.7,44.1


In [ ]:
# H0 =  There is no difference in the average packing times of the two machines, or the new machine is slower.
# H1 = The new machine packs faster on average than the current machine.



In [23]:
# Perform the t-test
t_statistic, two_tailed_p_value = ttest_ind(new_machine_times, current_machine_times)
t_statistic, two_tailed_p_value

(-3.3972307061176026, 0.0032111425007745158)

In [25]:
# Adjust for one-tailed test
one_tailed_p_value = two_tailed_p_value / 2
one_tailed_p_value 

0.0016055712503872579

In [22]:
# Interpret the results
if one_tailed_p_value < 0.05 and new_machine_times.mean() < current_machine_times.mean():
    print("Reject the null hypothesis: The new machine is faster.")
else:
    print("Cannot reject the null hypothesis: No sufficient evidence to show the new machine is faster.")

Reject the null hypothesis: The new machine is faster.


In [ ]:
# i dont know what type markdown thing means

#### Import dataset

In this challenge we will work on the Pokemon dataset you have already used. The goal is to test whether different groups of pokemon (e.g. Legendary vs Normal, Generation 1 vs 2, single-type vs dual-type) have different stats (e.g. HP, Attack, Defense, etc.). Use pokemon.csv

In [30]:
# Your code here:
pokemon_df = pd.read_csv("pokemon.txt")
pokemon_df

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True


#### First we want to define a function with which we can test the means of a feature set of two samples. 

In the next cell you'll see the annotations of the Python function that explains what this function does and its arguments and returned value. This type of annotation is called **docstring** which is a convention used among Python developers. The docstring convention allows developers to write consistent tech documentations for their codes so that others can read. It also allows some websites to automatically parse the docstrings and display user-friendly documentations.

Follow the specifications of the docstring and complete the function.

In [29]:
def t_test_features(s1, s2, features=['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'Total']):
    """Test means of a feature set of two samples
    
    Args:
        s1 (dataframe): sample 1
        s2 (dataframe): sample 2
        features (list): an array of features to test
    
    Returns:
        dict: a dictionary of t-test scores for each feature where the feature name is the key and the p-value is the value
    """
    results = {}

    # Your code here
     # Perform t-test for each feature
    for feature in features:
        # Perform the t-test and store the p-value in the dictionary
        stat, p_value = ttest_ind(s1[feature], s2[feature])
        results[feature] = p_value
    
    return results

#### Using the `t_test_features` function, conduct t-test for Lengendary vs non-Legendary pokemons.

*Hint: your output should look like below:*

```
{'HP': 1.0026911708035284e-13,
 'Attack': 2.520372449236646e-16,
 'Defense': 4.8269984949193316e-11,
 'Sp. Atk': 1.5514614112239812e-21,
 'Sp. Def': 2.2949327864052826e-15,
 'Speed': 1.049016311882451e-18,
 'Total': 9.357954335957446e-47}
 ```

In [31]:
# Your code here
# Split the data into Legendary and non-Legendary samples
legendary_pokemon = pokemon_df[pokemon_df['Legendary'] == True]
non_legendary_pokemon = pokemon_df[pokemon_df['Legendary'] == False]

In [32]:
legendary_pokemon

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
156,144,Articuno,Ice,Flying,580,90,85,100,95,125,85,1,True
157,145,Zapdos,Electric,Flying,580,90,90,85,125,90,100,1,True
158,146,Moltres,Fire,Flying,580,90,100,90,125,85,90,1,True
162,150,Mewtwo,Psychic,NaN,680,106,110,90,154,90,130,1,True
163,150,MewtwoMega Mewtwo X,Psychic,Fighting,780,106,190,100,154,100,130,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True


In [33]:
non_legendary_pokemon

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
787,711,GourgeistSuper Size,Ghost,Grass,494,85,100,122,58,75,54,6,False
788,712,Bergmite,Ice,NaN,304,55,69,85,32,35,28,6,False
789,713,Avalugg,Ice,NaN,514,95,117,184,44,46,28,6,False
790,714,Noibat,Flying,Dragon,245,40,30,35,45,40,55,6,False


In [34]:
t_test_results = t_test_features(legendary_pokemon, non_legendary_pokemon)

In [35]:
t_test_results

{'HP': 3.330647684846191e-15,
 'Attack': 7.827253003205333e-24,
 'Defense': 1.5842226094427259e-12,
 'Sp. Atk': 6.314915770427266e-41,
 'Sp. Def': 1.8439809580409594e-26,
 'Speed': 2.3540754436898437e-21,
 'Total': 3.0952457469652825e-52}

#### From the test results above, what conclusion can you make? Do Legendary and non-Legendary pokemons have significantly different stats on each feature?

In [ ]:
# Your comment here
# all the values are almost 0 so i would say that there is  a very significant difference on the stats on each features

#### Next, conduct t-test for Generation 1 and Generation 2 pokemons.

In [36]:
# Your code here
# filter the data 
gen1_pokemon = pokemon_df[pokemon_df['Generation'] == 1]
gen2_pokemon = pokemon_df[pokemon_df['Generation'] == 2]

# use the t_test features function
t_test_gen_results = t_test_features(gen1_pokemon, gen2_pokemon)

print(t_test_gen_results)

{'HP': 0.13791881412813622, 'Attack': 0.24050968418101457, 'Defense': 0.5407630349194362, 'Sp. Atk': 0.14119788176331508, 'Sp. Def': 0.16781226231606386, 'Speed': 0.00283569548125787, 'Total': 0.5599140649014442}


#### What conclusions can you make?

In [ ]:
# Your comment here
# almost all the values are higher than 0.05 ,the only one that is not is t speed, so i would say there is no significant differences between generation 1 and 2 pokemon ,excpet for speed,so either generation 1 or 2 are faster pokemons.

#### Compare pokemons who have single type vs those having two types.

In [38]:
# Your code here
single_type_pokemon = pokemon_df[pokemon_df['Type 2'].isna()]
dual_type_pokemon = pokemon_df[~pokemon_df['Type 2'].isna()]

t_test_type_results = t_test_features(single_type_pokemon, dual_type_pokemon)

print(t_test_type_results)

{'HP': 0.11060643144431842, 'Attack': 0.00015741395666164396, 'Defense': 3.250594205757004e-08, 'Sp. Atk': 0.0001454917404035147, 'Sp. Def': 0.00010893304795534396, 'Speed': 0.024051410794037463, 'Total': 1.1749035008828752e-07}


#### What conclusions can you make?

### Your comment here
* the result suggest that dual- type pokemon tend to have higer stats in serveral key areas compared to single type pokemon ,  which could be reflective on the additional advanteges of being a two type pokemon

#### Now, we want to compare whether there are significant differences of `Attack` vs `Defense`  and  `Sp. Atk` vs `Sp. Def` of all pokemons. Please write your code below.

*Hint: are you comparing different populations or the same population?*

In [42]:
# Your code here
attack_defense_pvalue = ttest_rel(pokemon_df['Attack'], pokemon_df['Defense']).pvalue
sp_atk_def_pvalue = ttest_rel(pokemon_df['Sp. Atk'], pokemon_df['Sp. Def']).pvalue

attack_defense_pvalue

1.7140303479358558e-05

In [43]:
sp_atk_def_pvalue

0.3933685997548122

#### What conclusions can you make?

###  Your comment here
* while there is a significant difference between Attack and Defense stats across Pokémon, there is no significant difference between Special Attack and Special Defense stats. This could imply that Pokémon are designed with a more distinct variation in their physical offensive and defensive capabilities, while their special abilities (Special Attack and Special Defense) are more balanced